# Weather Dataset Preprocessing and feature Engineering

In [12]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Using cached version of Emiasd-Flight-Data-Analysis.jar


error: error while loading FlightDataLoader, class file '/tmp/toree-tmp-dir2537429693683601932/toree_add_jars/Emiasd-Flight-Data-Analysis.jar(com/flightdelay/data/loaders/FlightDataLoader.class)' has location not matching its contents: contains class FlightDataLoader


Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

val rawFlightsPath = s"${configuration.common.output.basePath}/common/data/raw_flights.parquet"
val flightsDF = spark.read.parquet(rawFlightsPath)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@2a0c6123
session = org.apache.spark.sql.SparkSession@2a0c6123
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovya...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/*.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/*.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileConfig(/home/jovya...

In [7]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val DEFAULT_DATE_FORMAT = "yyyy-MM-dd"
def expectedSchema: StructType = StructType(Array(
    StructField("FL_DATE", StringType, nullable = false),
    StructField("OP_CARRIER_AIRLINE_ID", IntegerType, nullable = false),
    StructField("OP_CARRIER_FL_NUM", IntegerType, nullable = false),
    StructField("ORIGIN_AIRPORT_ID", IntegerType, nullable = false),
    StructField("DEST_AIRPORT_ID", IntegerType, nullable = false),
    StructField("CRS_DEP_TIME", IntegerType, nullable = false),
    StructField("ARR_DELAY_NEW", DoubleType, nullable = true),
    StructField("CANCELLED", IntegerType, nullable = true),
    StructField("DIVERTED", IntegerType, nullable = true),
    StructField("CRS_ELAPSED_TIME", DoubleType, nullable = true),
    StructField("WEATHER_DELAY", DoubleType, nullable = true),
    StructField("NAS_DELAY", DoubleType, nullable = true)
  ))

val csvRawFlightsPath = s"${configuration.common.data.basePath}/FLIGHT-3Y/Flights/*.csv"
val df = spark.read.format("csv")
  .option("header", "true")
  .schema(expectedSchema)
  .option("timestampFormat", DEFAULT_DATE_FORMAT)
  .option("multiline", "true")
  .option("escape", "\"")
  .load(csvRawFlightsPath)
  .withColumn("FL_DATE", to_date(col("FL_DATE"), DEFAULT_DATE_FORMAT))

DEFAULT_DATE_FORMAT = yyyy-MM-dd
csvRawFlightsPath = /home/jovyan/work/data/FLIGHT-3Y/Flights/*.csv
df = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


expectedSchema: org.apache.spark.sql.types.StructType


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]

In [8]:
df.count()

18286055

In [3]:
flightsDF.count()

18286055

In [4]:
flightsDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)



In [5]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder, VectorAssembler}
import scala.util.Try

In [8]:
flightsDF.show(10)

+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+
|   FL_DATE|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|CRS_DEP_TIME|ARR_DELAY_NEW|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|WEATHER_DELAY|NAS_DELAY|
+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+
|2013-07-01|                20363|             3407|            11433|          13342|        1040|          0.0|     NULL|    NULL|            79.0|         NULL|     NULL|
|2013-07-01|                20363|             3409|            11433|          12266|        1227|          0.0|     NULL|    NULL|           175.0|         NULL|     NULL|
|2013-07-01|                20363|             3848|            13485|          11433|         900|          2.0|     NULL|    NUL

In [30]:
flightsDF.select("CANCELLED").distinct().show()

+---------+
|CANCELLED|
+---------+
|     NULL|
+---------+



In [31]:
flightsDF.select("DIVERTED").distinct().show()

+--------+
|DIVERTED|
+--------+
|    NULL|
+--------+



## 1. OP_CARRIER_AIRLINE_ID

In [8]:
flightsDF.select("OP_CARRIER_AIRLINE_ID").distinct().count()

15

In [7]:
flightsDF.select("OP_CARRIER_AIRLINE_ID").distinct().show(20)

+---------------------+
|OP_CARRIER_AIRLINE_ID|
+---------------------+
|                20398|
|                19690|
|                20409|
|                20378|
|                19790|
|                20304|
|                20436|
|                19393|
|                20355|
|                19805|
|                19930|
|                20366|
|                19977|
|                21171|
|                20437|
+---------------------+



In [11]:
flightsDF.select("OP_CARRIER_FL_NUM").distinct().count()

6670

In [10]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("OP_CARRIER_AIRLINE_ID")
  .agg(countDistinct("OP_CARRIER_FL_NUM").as("nb_flights_distinct"))
  .orderBy(desc("nb_flights_distinct"))
  .show(50, false)

+---------------------+-------------------+
|OP_CARRIER_AIRLINE_ID|nb_flights_distinct|
+---------------------+-------------------+
|19393                |4946               |
|19790                |2100               |
|19805                |2056               |
|20366                |1961               |
|20437                |1750               |
|20398                |1703               |
|20355                |1671               |
|20304                |1664               |
|19977                |1580               |
|20409                |1028               |
|20436                |649                |
|19930                |618                |
|20378                |434                |
|19690                |318                |
|21171                |254                |
+---------------------+-------------------+



In [9]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM")
  .agg(count("*").as("nb_vols"))
  .orderBy(desc("nb_vols"))
  .show(50, false)

+---------------------+-----------------+-------+
|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|nb_vols|
+---------------------+-----------------+-------+
|19930                |64               |1820   |
|19930                |65               |1815   |
|19930                |62               |1809   |
|19393                |24               |1580   |
|19930                |55               |1456   |
|19930                |61               |1448   |
|19930                |66               |1448   |
|19930                |67               |1416   |
|19393                |429              |1351   |
|19393                |47               |1327   |
|19393                |6                |1284   |
|19393                |1015             |1281   |
|19393                |910              |1251   |
|19393                |40               |1206   |
|19393                |1983             |1199   |
|19393                |132              |1152   |
|19393                |38               |1151   |


## 2. ORIGIN_AIRPORT_ID et DEST_AIRPORT_ID

In [14]:
flightsDF.select("ORIGIN_AIRPORT_ID").distinct().count()

312

In [15]:
flightsDF.select("DEST_AIRPORT_ID").distinct().count()

313

In [17]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("ORIGIN_AIRPORT_ID")
  .agg(countDistinct("OP_CARRIER_FL_NUM").as("nb_flights_distinct"))
  .orderBy(desc("nb_flights_distinct"))
  .show(20, false)

+-----------------+-------------------+
|ORIGIN_AIRPORT_ID|nb_flights_distinct|
+-----------------+-------------------+
|11292            |4487               |
|12889            |3928               |
|12892            |3759               |
|14107            |3623               |
|10821            |3532               |
|13232            |3504               |
|10397            |3480               |
|13930            |3470               |
|14771            |2941               |
|13204            |2789               |
|14679            |2773               |
|15016            |2756               |
|13198            |2697               |
|10693            |2681               |
|11618            |2527               |
|12191            |2513               |
|11259            |2500               |
|12266            |2482               |
|11433            |2467               |
|11298            |2433               |
+-----------------+-------------------+
only showing top 20 rows



In [18]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("DEST_AIRPORT_ID")
  .agg(countDistinct("OP_CARRIER_FL_NUM").as("nb_flights_distinct"))
  .orderBy(desc("nb_flights_distinct"))
  .show(20, false)

+---------------+-------------------+
|DEST_AIRPORT_ID|nb_flights_distinct|
+---------------+-------------------+
|11292          |4465               |
|12889          |3838               |
|12892          |3767               |
|14107          |3567               |
|13232          |3526               |
|10821          |3503               |
|13930          |3461               |
|10397          |3411               |
|14771          |2920               |
|13204          |2755               |
|15016          |2693               |
|14679          |2630               |
|13198          |2546               |
|12191          |2529               |
|10693          |2516               |
|11618          |2506               |
|11259          |2467               |
|12266          |2398               |
|11298          |2387               |
|11433          |2349               |
+---------------+-------------------+
only showing top 20 rows



## 3. CRS_ELAPSED_TIME

In [19]:
flightsDF.select("CRS_ELAPSED_TIME").distinct().count()

504

In [21]:
flightsDF.select("CRS_ELAPSED_TIME").distinct().show(20)

+----------------+
|CRS_ELAPSED_TIME|
+----------------+
|           305.0|
|           299.0|
|           170.0|
|           184.0|
|           147.0|
|           160.0|
|           169.0|
|            70.0|
|            67.0|
|           379.0|
|           311.0|
|           168.0|
|           206.0|
|            69.0|
|           389.0|
|           390.0|
|           365.0|
|           249.0|
|           142.0|
|           191.0|
+----------------+
only showing top 20 rows



In [23]:
flightsDF
  .select("CRS_ELAPSED_TIME")
  .summary("count", "min", "25%", "50%", "75%", "max", "mean", "stddev")
  .show()

+-------+-----------------+
|summary| CRS_ELAPSED_TIME|
+-------+-----------------+
|  count|          6096761|
|    min|              1.0|
|    25%|             81.0|
|    50%|            115.0|
|    75%|            165.0|
|    max|           1865.0|
|   mean|133.9327569179766|
| stddev|72.77188749215703|
+-------+-----------------+



In [32]:
import org.apache.spark.sql.functions._

// Récupérer les quantiles
val quantiles = flightsDF.stat.approxQuantile("CRS_ELAPSED_TIME", Array(0.25, 0.75), 0.0)
val q1 = quantiles(0)
val q3 = quantiles(1)
val iqr = q3 - q1

val lowerBound = q1 - 1.5 * iqr
val upperBound = q3 + 1.5 * iqr

println(s"Bornes acceptées : $lowerBound à $upperBound")

// Filtrer les données
val flightsNoOutliers = flightsDF.filter(
  col("CRS_ELAPSED_TIME") >= lowerBound && col("CRS_ELAPSED_TIME") <= upperBound
)

Bornes acceptées : -45.0 à 291.0


quantiles = Array(81.0, 165.0)
q1 = 81.0
q3 = 165.0
iqr = 84.0
lowerBound = -45.0
upperBound = 291.0
flightsNoOutliers = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]

# Null Values 

In [6]:
import org.apache.spark.sql.functions._

val totalCount = flightsDF.count()

val nullCountsExprs = flightsDF.columns.map(c => sum(when(col(c).isNull, 1).otherwise(0)).alias(c))

val nullCountsRow = flightsDF.agg(nullCountsExprs.head, nullCountsExprs.tail: _*).first()

val nullStats = flightsDF.columns.map { c =>
  val nullCount = nullCountsRow.getAs[Long](c)
  val percent = (nullCount.toDouble / totalCount) * 100
  (c, nullCount, f"$percent%.2f%%")
}

import spark.implicits._
val nullStatsDF = nullStats.toSeq.toDF("Colonne", "Valeurs_nulles", "Pourcentage")

nullStatsDF.show(false)

+---------------------+--------------+-----------+
|Colonne              |Valeurs_nulles|Pourcentage|
+---------------------+--------------+-----------+
|FL_DATE              |0             |0.00%      |
|OP_CARRIER_AIRLINE_ID|0             |0.00%      |
|OP_CARRIER_FL_NUM    |0             |0.00%      |
|ORIGIN_AIRPORT_ID    |0             |0.00%      |
|DEST_AIRPORT_ID      |0             |0.00%      |
|CRS_DEP_TIME         |1             |0.00%      |
|ARR_DELAY_NEW        |342986        |1.88%      |
|CANCELLED            |18286055      |100.00%    |
|DIVERTED             |18286055      |100.00%    |
|CRS_ELAPSED_TIME     |12            |0.00%      |
|WEATHER_DELAY        |14761092      |80.72%     |
|NAS_DELAY            |14761092      |80.72%     |
+---------------------+--------------+-----------+



totalCount = 18286055
nullCountsExprs = Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID, sum(CASE WHEN (CRS_DEP_TIME IS NULL) THEN 1 ELSE 0 END) AS CRS_DEP_TIME, sum(CASE WHEN (ARR_DELAY_NEW IS NULL) THEN 1 ELSE 0 END) AS ARR_DELAY_NEW, sum(CASE WHEN (CANCELLED IS NULL) THEN 1 ELSE 0 END) AS CANCELLED, sum(CASE WHEN (DIVERTED IS NULL) THEN 1 ELSE 0 END) AS ...


Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID, sum(CASE WHEN (CRS_DEP_TIME IS NULL) THEN 1 ELSE 0 END) AS CRS_DEP_TIME, sum(CASE WHEN (ARR_DELAY_NEW IS NULL) THEN 1 ELSE 0 END) AS ARR_DELAY_NEW, sum(CASE WHEN (CANCELLED IS NULL) THEN 1 ELSE 0 END) AS CANCELLED, sum(CASE WHEN (DIVERTED IS NULL) THEN 1 ELSE 0 END) AS ...

In [9]:
import org.apache.spark.sql.functions._

val totalCount = df.count()

val nullCountsExprs = df.columns.map(c => sum(when(col(c).isNull, 1).otherwise(0)).alias(c))

val nullCountsRow = df.agg(nullCountsExprs.head, nullCountsExprs.tail: _*).first()

val nullStats = df.columns.map { c =>
  val nullCount = nullCountsRow.getAs[Long](c)
  val percent = (nullCount.toDouble / totalCount) * 100
  (c, nullCount, f"$percent%.2f%%")
}

import spark.implicits._
val nullStatsDF = nullStats.toSeq.toDF("Colonne", "Valeurs_nulles", "Pourcentage")

nullStatsDF.show(false)

+---------------------+--------------+-----------+
|Colonne              |Valeurs_nulles|Pourcentage|
+---------------------+--------------+-----------+
|FL_DATE              |0             |0.00%      |
|OP_CARRIER_AIRLINE_ID|0             |0.00%      |
|OP_CARRIER_FL_NUM    |0             |0.00%      |
|ORIGIN_AIRPORT_ID    |0             |0.00%      |
|DEST_AIRPORT_ID      |0             |0.00%      |
|CRS_DEP_TIME         |1             |0.00%      |
|ARR_DELAY_NEW        |342986        |1.88%      |
|CANCELLED            |18286055      |100.00%    |
|DIVERTED             |18286055      |100.00%    |
|CRS_ELAPSED_TIME     |12            |0.00%      |
|WEATHER_DELAY        |14761092      |80.72%     |
|NAS_DELAY            |14761092      |80.72%     |
+---------------------+--------------+-----------+



totalCount = 18286055
nullCountsExprs = Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID, sum(CASE WHEN (CRS_DEP_TIME IS NULL) THEN 1 ELSE 0 END) AS CRS_DEP_TIME, sum(CASE WHEN (ARR_DELAY_NEW IS NULL) THEN 1 ELSE 0 END) AS ARR_DELAY_NEW, sum(CASE WHEN (CANCELLED IS NULL) THEN 1 ELSE 0 END) AS CANCELLED, sum(CASE WHEN (DIVERTED IS NULL) THEN 1 ELSE 0 END) AS ...


Array(sum(CASE WHEN (FL_DATE IS NULL) THEN 1 ELSE 0 END) AS FL_DATE, sum(CASE WHEN (OP_CARRIER_AIRLINE_ID IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_AIRLINE_ID, sum(CASE WHEN (OP_CARRIER_FL_NUM IS NULL) THEN 1 ELSE 0 END) AS OP_CARRIER_FL_NUM, sum(CASE WHEN (ORIGIN_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS ORIGIN_AIRPORT_ID, sum(CASE WHEN (DEST_AIRPORT_ID IS NULL) THEN 1 ELSE 0 END) AS DEST_AIRPORT_ID, sum(CASE WHEN (CRS_DEP_TIME IS NULL) THEN 1 ELSE 0 END) AS CRS_DEP_TIME, sum(CASE WHEN (ARR_DELAY_NEW IS NULL) THEN 1 ELSE 0 END) AS ARR_DELAY_NEW, sum(CASE WHEN (CANCELLED IS NULL) THEN 1 ELSE 0 END) AS CANCELLED, sum(CASE WHEN (DIVERTED IS NULL) THEN 1 ELSE 0 END) AS ...

# D1, D2, D3 et D4

In [17]:
import com.flightdelay.data.preprocessing.DataPreprocessor
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Column, DataFrame, SparkSession}

/**
 * Génère des colonnes binaires D1, D2_<seuil>, D3, D4 (1 ou 0) selon la logique métier :
 *
 * - D1 : vols dont le retard total est imputé *seulement* à WEATHER et/ou NAS
 *        (ARR_DELAY_NEW ≈ WEATHER_DELAY + NAS_DELAY à epsilon près, et ARR_DELAY_NEW > 0)
 * - D2_<T> : vols avec WEATHER_DELAY > 0 et NAS_DELAY >= T
 * - D3 : vols avec WEATHER_DELAY > 0 ou NAS_DELAY > 0
 * - D4 : tous les vols avec ARR_DELAY_NEW > 0
 *
 * Nulls gérés via coalesce(..., 0.0)
 */
object FlightDataSetFilterGenerator {

  private val D2_THRESHOLDS_MINUTES: Seq[Int] = Seq(15, 30, 45, 60, 90)
  private val EPSILON_EQUALITY_MIN = 0.5

  private def nzDouble(colName: String): Column =
    coalesce(col(colName).cast("double"), lit(0.0))

  private def validateRequiredColumns(df: DataFrame): Unit = {
    val required = Seq("ARR_DELAY_NEW", "WEATHER_DELAY", "NAS_DELAY")
    val missing = required.filterNot(df.columns.contains)
    require(
      missing.isEmpty,
      s"Colonnes manquantes: ${missing.mkString(", ")} (requis: ${required.mkString(", ")})"
    )
  }

  /**
   * Ajoute les colonnes D1, D2_<seuil>, D3, D4 en binaire (1 / 0)
   */
  def withDelayFilters(df: DataFrame, epsilon: Double = EPSILON_EQUALITY_MIN): DataFrame = {
    validateRequiredColumns(df)

    val arr  = nzDouble("ARR_DELAY_NEW")
    val wthr = nzDouble("WEATHER_DELAY")
    val nas  = nzDouble("NAS_DELAY")

    val d4 = when(arr > 0, 1).otherwise(0)
    val d3 = when((wthr > 0) or (nas > 0), 1).otherwise(0)
    val d1 = when((arr > 0) && (abs(arr - (wthr + nas)) <= epsilon), 1).otherwise(0)

    // Appliquer les colonnes de base
    val baseDf = df
      .withColumn("D4", d4)
      .withColumn("D3", d3)
      .withColumn("D1", d1)

    // Ajouter D2_<seuil>
    val dfWithD2 = D2_THRESHOLDS_MINUTES.foldLeft(baseDf) { (acc, thr) =>
      val colName = s"D2_$thr"
      val d2col = when((wthr > 0) && (nas >= thr), 1).otherwise(0)
      acc.withColumn(colName, d2col)
    }

    dfWithD2
  }

  /**
   * Méthode exécutable pour test (facultative)
   */
  def run(spark: SparkSession, input: DataFrame): DataFrame = {
    val enriched = withDelayFilters(input)

    val total = enriched.count()
    println(s"[FlightDataSetFilterGenerator] Total vols: $total")

    val colsToLog = Seq("D1", "D3", "D4") ++ D2_THRESHOLDS_MINUTES.map(t => s"D2_$t")
    colsToLog.foreach { c =>
      val n = enriched.filter(col(c) === 1).count()
      val pct = if (total > 0) (n.toDouble / total * 100) else 0.0
      println(f" - $c%-6s : $n%8d vols (${pct}%.2f%%)")
    }

    enriched
  }
}

defined object FlightDataSetFilterGenerator


In [21]:
val extendedFlightsDF = FlightDataSetFilterGenerator.run(spark, flightsDF)

df2.show(10)

[FlightDataSetFilterGenerator] Total vols: 18286055
 - D1     :   710294 vols (3.88%)
 - D3     :  2024941 vols (11.07%)
 - D4     :  7017890 vols (38.38%)
 - D2_15  :    53900 vols (0.29%)
 - D2_30  :    26658 vols (0.15%)
 - D2_45  :    14062 vols (0.08%)
 - D2_60  :     7797 vols (0.04%)
 - D2_90  :     2570 vols (0.01%)
+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+---+---+---+-----+-----+-----+-----+-----+
|   FL_DATE|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|CRS_DEP_TIME|ARR_DELAY_NEW|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|WEATHER_DELAY|NAS_DELAY| D4| D3| D1|D2_15|D2_30|D2_45|D2_60|D2_90|
+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+---+---+---+-----+-----+-----+-----+-----+
|2013-07-01|              

extendedFlightsDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 18 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 18 more fields]

In [22]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, SparkSession, Column}

/**
 * Construit un train/test équilibré (50/50 delayed vs on-time) en filtrant d'abord sur une colonne Dx (=1).
 *
 * Procédure :
 * 1) Filtre df sur dxCol == 1 (optionnellement désactivable).
 * 2) Construit la label col "is_delayed" = (ARR_DELAY_NEW >= delayThresholdMin).
 * 3) Sépare delayed / on-time.
 * 4) Split delayed en train/test avec ratio 3:1.
 * 5) Échantillonne aléatoirement (sans répétition) on-time pour avoir le même nombre que delayed dans train et test.
 * 6) Renvoie (trainDF, testDF), mélangés.
 */
object DelayBalancedDatasetBuilder {

  /** Sélectionne N lignes aléatoires sans répétition. Si N > count(df), prend tout. */
  private def sampleN(df: DataFrame, n: Long, seed: Long): DataFrame = {
    val total = df.count()
    val take = math.min(n, total)
    if (take <= 0) df.limit(0)
    else df.withColumn("__rnd", rand(seed)).orderBy(col("__rnd")).limit(take.toInt).drop("__rnd")
  }

  /** Vérifie les colonnes nécessaires. */
  private def validate(df: DataFrame, dxCol: String): Unit = {
    val needed = Seq("ARR_DELAY_NEW", dxCol)
    val missing = needed.filterNot(df.columns.contains)
    require(missing.isEmpty, s"Colonnes manquantes: ${missing.mkString(", ")}")
  }

  /**
   * @param df                DataFrame d’origine (doit contenir ARR_DELAY_NEW et dxCol)
   * @param dxCol             Colonne binaire 1/0 (ex: "D1", "D2_60", "D3", "D4")
   * @param delayThresholdMin Seuil pour définir "delayed" (ex: 15, 60)
   * @param filterOnDxEquals1 Si true, on restreint aux lignes où dxCol == 1 avant équilibrage
   * @param trainRatio        Part de delayed envoyée en train (ex: 0.75 ~ 3:1)
   * @param seed              Graine de random
   * @return (trainBalanced, testBalanced)
   */
  def buildBalancedTrainTest(
      df: DataFrame,
      dxCol: String,
      delayThresholdMin: Int,
      filterOnDxEquals1: Boolean = true,
      trainRatio: Double = 0.75,
      seed: Long = 42L
  ): (DataFrame, DataFrame) = {

    validate(df, dxCol)

    val base = if (filterOnDxEquals1) df.filter(col(dxCol) === 1) else df

    // Label binaire : 1 si retard >= seuil, 0 sinon
    val labeled = base.withColumn(
      "is_delayed",
      when(coalesce(col("ARR_DELAY_NEW").cast("double"), lit(0.0)) >= delayThresholdMin, 1).otherwise(0)
    )

    val delayed   = labeled.filter(col("is_delayed") === 1)
    val onTime    = labeled.filter(col("is_delayed") === 0)

    // Split des "delayed" en 3:1 (train:test)
    val Array(trainDelayed, testDelayed) = delayed.randomSplit(Array(trainRatio, 1.0 - trainRatio), seed)

    val nTrainDelayed = trainDelayed.count()
    val nTestDelayed  = testDelayed.count()

    // Sous-échantillonnage on-time pour égaliser
    val trainOnTime = sampleN(onTime, nTrainDelayed, seed + 1)
    val testOnTime  = sampleN(onTime.except(trainOnTime), nTestDelayed, seed + 2)

    // Union + shuffle
    val trainBalanced = trainDelayed.unionByName(trainOnTime).withColumn("__rnd", rand(seed + 3)).orderBy(col("__rnd")).drop("__rnd")
    val testBalanced  = testDelayed.unionByName(testOnTime).withColumn("__rnd", rand(seed + 4)).orderBy(col("__rnd")).drop("__rnd")

    // (Optionnel) Log rapide
    def logSplit(name: String, d: DataFrame): Unit = {
      val total = d.count()
      val nDel  = d.filter(col("is_delayed") === 1).count()
      val nOn   = d.filter(col("is_delayed") === 0).count()
      println(f"[$name] total=$total%8d | delayed=$nDel%8d | on-time=$nOn%8d")
    }
    logSplit(s"TRAIN ($dxCol, t=$delayThresholdMin)", trainBalanced)
    logSplit(s"TEST  ($dxCol, t=$delayThresholdMin)", testBalanced)

    (trainBalanced, testBalanced)
  }

  /** Variante simple : renvoie un unique DataFrame équilibré (sans split train/test). */
  def buildBalancedSingle(
      df: DataFrame,
      dxCol: String,
      delayThresholdMin: Int,
      filterOnDxEquals1: Boolean = true,
      seed: Long = 42L
  ): DataFrame = {
    validate(df, dxCol)
    val base = if (filterOnDxEquals1) df.filter(col(dxCol) === 1) else df
    val labeled = base.withColumn(
      "is_delayed",
      when(coalesce(col("ARR_DELAY_NEW").cast("double"), lit(0.0)) >= delayThresholdMin, 1).otherwise(0)
    )
    val delayed = labeled.filter(col("is_delayed") === 1)
    val onTime  = labeled.filter(col("is_delayed") === 0)
    val n = math.min(delayed.count(), onTime.count())
    val delayedS = sampleN(delayed, n, seed)
    val onTimeS  = sampleN(onTime, n, seed + 1)
    delayedS.unionByName(onTimeS).withColumn("__rnd", rand(seed + 2)).orderBy(col("__rnd")).drop("__rnd")
  }
}

defined object DelayBalancedDatasetBuilder


In [25]:
// Exemple : équilibrer sur D2_60, seuil "delayed" = 60 minutes
val (trainDF, testDF) =
  DelayBalancedDatasetBuilder.buildBalancedTrainTest(
    df = extendedFlightsDF,
    dxCol = "D2_60",
    delayThresholdMin = 60, // ou 15, 30, 45, 90
    filterOnDxEquals1 = true, // garde seulement les vols du groupe Dx
    trainRatio = 0.75,
    seed = 2025L
  )

// Vérif rapide
trainDF.groupBy("is_delayed").count().show()
testDF.groupBy("is_delayed").count().show()

// Vérif rapide
trainDF.show(5)
testDF.show(5)

[TRAIN (D2_60, t=60)] total=    5859 | delayed=    5859 | on-time=       0
[TEST  (D2_60, t=60)] total=    1938 | delayed=    1938 | on-time=       0
+----------+-----+
|is_delayed|count|
+----------+-----+
|         1| 5859|
+----------+-----+

+----------+-----+
|is_delayed|count|
+----------+-----+
|         1| 1938|
+----------+-----+

+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+---+---+---+-----+-----+-----+-----+-----+----------+
|   FL_DATE|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|CRS_DEP_TIME|ARR_DELAY_NEW|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|WEATHER_DELAY|NAS_DELAY| D4| D3| D1|D2_15|D2_30|D2_45|D2_60|D2_90|is_delayed|
+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+---+---+---+-----+-----+-----+-

trainDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 19 more fields]
testDF = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 19 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 19 more fields]